# LDA over Time

In [41]:
from utils import * 
import plotly.plotly as py
import plotly 
import time
import datetime
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
plotly.tools.set_credentials_file(username='kt426', api_key='GcKIXRGajqJ9KlQAcL2L')

### Load dataframe

In [5]:
DATAPASS = '../../../yelp10/dataset/business.json'
business = load_json_to_df(DATAPASS)

In [11]:
DATAPASS2 = '../../../yelp10/dataset/review.json'
review = load_json_to_df(DATAPASS2)

In [6]:
categories = {}
for C in business.categories:
    for c in C:
        if c in categories:
            categories[c] += 1
        else:
            categories[c] = 1

In [7]:
categories = sorted(categories.items(), key=lambda x: x[1], reverse = True)

In [13]:
categories[:20]

[('Restaurants', 51613),
 ('Shopping', 24595),
 ('Food', 23014),
 ('Beauty & Spas', 15139),
 ('Home Services', 13202),
 ('Health & Medical', 12033),
 ('Nightlife', 11364),
 ('Bars', 9868),
 ('Automotive', 9476),
 ('Local Services', 9343),
 ('Event Planning & Services', 8038),
 ('Active Life', 7427),
 ('Fashion', 6299),
 ('Sandwiches', 5864),
 ('Fast Food', 5792),
 ('American (Traditional)', 5737),
 ('Pizza', 5652),
 ('Coffee & Tea', 5565),
 ('Hair Salons', 5395),
 ('Hotels & Travel', 5188)]

In [32]:
review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


In [29]:
def business_id_retrieval(cat, business):
    '''
    
    Input:
        cat(str) : category
        business(dataframe) : the business data
    Output:
        id_list(set) : business ids of a particular category
    '''
    id_list = set()
    idx = 0
    for row in business.values:
        categories = row[3]
        if cat in categories:
            id_list.add(row[2])
    return id_list

## Experiment with Pizza

In [31]:
pizza = business_id_retrieval('Pizza', business)

In [33]:
review_pizza = review.loc[review['business_id'].isin(pizza)]

### Observe review count over time

In [40]:
import plotly.graph_objs as go
data = [go.Scatter(x=review_pizza.groupby('date').count()['review_id'].keys()\
                   ,y=review_pizza.groupby('date').count()['review_id'].values)]
layout = go.Layout( title='Piazza Reviews',
    xaxis={'title':'Date'},
    yaxis={'title':'Number of reviews'},
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Later, we need to normalize the growth factor

### Investigate review text, tf-idf and LDA

In [42]:
vectorizer = CountVectorizer(max_df = 0.3)
doc = vectorizer.fit_transform(review_pizza['text'])

This takes forever, don't do it on your machine

In [43]:
freqs1 = [(word, doc.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]
#sort from largest to smallest
print (sorted (freqs1, key = lambda x: -x[1])[:20])

KeyboardInterrupt: 

In [53]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(review_pizza['text'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

This also takes long time, we need an access to the server

In [55]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=50, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

In [56]:
'''
Thanks Eric Ma
'''
def display_topics(model, feature_names, n_top_words):
    for topic_index, topic in enumerate(model.components_):
        print("Topic %d:" % topic_index)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words:-1]]))

In [57]:
display_topics(nmf, tfidf_feature_names, 10)

Topic 0:
night late dinner went friday date saturday open wine
Topic 1:
pizza better hut places joint pepperoni price dough ny
Topic 2:
minutes table said wait came asked told server took
Topic 3:
order called phone online pick placed wrong customer told
Topic 4:
good pretty prices price overall bit decent value quite
Topic 5:
place eat looking clean people friends family favorite want
Topic 6:
ordered came large tasted half didn pepperoni wasn husband
Topic 7:
service customer slow fast horrible terrible bad friendly poor
Topic 8:
great atmosphere prices spot experience family fantastic service price
Topic 9:
staff friendly super helpful clean fast attentive wait family
Topic 10:
love favorite family absolutely kids coming come salads yummy
Topic 11:
fresh ingredients quality hot mozzarella tasty basil oven dough
Topic 12:
wings hot buffalo crispy ranch honey bbq fries medium
Topic 13:
und die der das ist sehr ich nicht war
Topic 14:
sauce garlic bread knots tomato red pasta marinara 